In [1]:
from fastai import *
from fastai.text import * 

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
df = pd.read_csv('data/quotes.csv', sep=r'\<\|\>', header=None, names=['text', 'author'])

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
df.head()

,text,author
0,A serious and good philosophical work could be...,Wittgenstein
1,The limits of my language means the limits of ...,Wittgenstein
2,"I don't know why we are here, but I'm pretty s...",Wittgenstein
3,"Whereof one cannot speak, thereof one must be ...",Wittgenstein
4,Hell isn't other people. Hell is yourself.,Wittgenstein


In [ ]:
df['text'] = df['text'].str.lower()

In [ ]:
valid_pct = 0.05 #validation percent
df = df.iloc[np.random.permutation(len(df))]
cut = int(valid_pct * len(df)) + 1
train_df, valid_df = df[cut:], df[:cut]

In [ ]:
len(train_df), len(valid_df)

(4873, 257)

In [ ]:
nan_rows = df[df['text'].isnull()]

In [ ]:
nan_rows

,text,author


In [ ]:
data_lm = TextLMDataBunch.from_df('data', train_df, valid_df, text_cols='text',
                           label_cols='author')

In [ ]:
# data_clas = TextClasDataBunch.from_df('data', train_df, valid_df, text_cols='text', label_cols='author', vocab=data_lm.train_ds.vocab, bs=32)

In [ ]:
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,4.763811,4.384274,0.224282


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,4.414166,4.182730,0.253518


In [ ]:
wd=1e-7
lr=1e-3
lrs = lr

In [ ]:
learn.fit(10,lrs, wd)

epoch,train_loss,valid_loss,accuracy
1,4.253211,4.066961,0.263676


In [ ]:
learn.predict("xxbos", n_words=50, temperature=0.75)

In [ ]:
number_of_ideas = 100
ideas_counter = 0
all_ideas = []

for i in range(1000):
    idea = learn.predict("xxbos xxfld 1", n_words=20, temperature=0.8)
    ideas = idea.split("xxbos xxfld 1")
    ideas = ideas[1:-1]
    
    for idea in ideas:
        idea = idea.replace("xxbos xxfld 1 ","").strip()
        if(idea):
            all_ideas.append(idea)
            ideas_counter = ideas_counter+1
            
    if ideas_counter > number_of_ideas:
        break

In [ ]:
all_ideas

In [ ]:
learn.save_encoder('ft_enc')

In [ ]:
train_df.to_pickle('data/train_df.pkl')

In [ ]:
valid_df.to_pickle('data/valid_df.pkl')